In [1]:
import pandas as pd
import numpy as np
import re
import json
import copy

In [2]:
df = pd.read_csv('books_1.Best_Books_Ever.csv')

In [3]:
columnsToDrop = ['bookId', 'isbn', 'firstPublishDate', 'ratingsByStars', 'likedPercent', 'bbeScore', 'bbeVotes',
                 'edition', 'awards']

In [4]:
for col in columnsToDrop:
    try:
        df.drop(columns=col, inplace=True)
    except:
        print(f"{col} not found")

In [5]:
def nanInColsOfDF(df_):
    cols_ = df_.columns
    for col_ in cols_:
        count_ = len(df_[df_[col_].isna()])
        print(f"{col_}, {count_}")

In [6]:
df = df.loc[df.coverImg.notna()]
df = df.loc[df.pages.notna()]
df = df.loc[df.description.notna()]
df = df.loc[df.language.notna()]
df = df.loc[df.bookFormat.notna()]
df = df.loc[df.publisher.notna()]
df = df.loc[df.publishDate.notna()]

In [7]:
df['series'] = df['series'].fillna("")
df['author'] = df['author'].str.replace('\(Goodreads Author\)', '', regex=True)
df['pages'] = df['pages'].str.replace('1 page', '0', regex=True)
df.pages = df.pages.astype(int)

a = df.author
a = a.str.split(',', expand=True)[0]
df.author = a

In [8]:
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

df['genres'] = df.genres.apply(lambda x: x[1:-1].split("', '"))
df['characters'] = df.characters.apply(lambda x: x[1:-1].split("', '"))
df['setting'] = df.setting.apply(lambda x: x[1:-1].split("', '"))

In [9]:
for i in range(len(df.genres)):
    for j in range(len(df.genres[i])):
        df.genres[i][j] = df.genres[i][j].replace("[", "")
        df.genres[i][j] = df.genres[i][j].replace("'", "")
        df.genres[i][j] = df.genres[i][j].replace(",", "")
        df.genres[i][j] = df.genres[i][j].replace("]", "")

for i in range(len(df.characters)):
    for j in range(len(df.characters[i])):
        df.characters[i][j] = df.characters[i][j].replace("[", "")
        df.characters[i][j] = df.characters[i][j].replace("'", "")
        df.characters[i][j] = df.characters[i][j].replace(",", "")
        df.characters[i][j] = df.characters[i][j].replace("]", "")

for i in range(len(df.setting)):
    for j in range(len(df.setting[i])):
        df.setting[i][j] = df.setting[i][j].replace("[", "")
        df.setting[i][j] = df.setting[i][j].replace("'", "")
        df.setting[i][j] = df.setting[i][j].replace(",", "")
        df.setting[i][j] = df.setting[i][j].replace("]", "")

In [10]:
for row in df.loc[df.price.notna()].price:
    val = row.count('.')
    if val == 2:
        print(row)
        df = df.loc[df.price.notna()].drop(df.loc[df.price.notna()].loc[df.loc[df.price.notna()].price == row].index)

df.price = df.price.astype(float)
df.price = df.price * 3
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

1.189.88
1.743.28
1.743.28
1.307.46
1.307.46
8.715.51
1.775.18
1.734.84
1.734.84
2.812.18


In [11]:
df['price'] = np.random.uniform(4, 30, df.shape[0]).round(2)

In [12]:
genres = []
for genList in df.genres:
    for gen in genList:
        if gen not in genres:
            genres.append(gen)
genres.sort()

setting = []
for settingList in df.setting:
    for loc in settingList:
        if loc not in setting:
            setting.append(loc)
setting.sort()

languages = []
for lang in df.language:
    if lang not in languages:
        languages.append(lang)
languages.sort()

authors = []
for auth in df.author:
    if auth not in authors:
        authors.append(auth)
authors.sort()

In [15]:
df.columns

Index(['title', 'series', 'author', 'rating', 'description', 'language',
       'genres', 'characters', 'bookFormat', 'pages', 'publisher',
       'publishDate', 'numRatings', 'setting', 'coverImg', 'price'],
      dtype='object')

In [16]:
df.rename(
    columns={'title': 'Title', 'series': 'Series', 'author': 'Author', 'rating': 'Rating', 'description': 'Description',
             'language': 'Language', 'genres': 'Genres', 'characters': 'Characters', 'bookFormat': 'BookFormat',
             'pages': 'Pages', 'publisher': 'Publisher', 'publishDate': 'PublishDate', 'numRatings': 'NumOfRatings',
             'setting': 'Setting', 'coverImg': 'CoverImg', 'price': 'Price'}, inplace=True)

In [17]:
df

,Title,Series,Author,Rating,Description,Language,Genres,Characters,BookFormat,Pages,Publisher,PublishDate,NumOfRatings,Setting,CoverImg,Price
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,"[Young Adult, Fiction, Dystopia, Fantasy, Scie...","[Katniss Everdeen, Peeta Mellark, Cato (Hunger...",Hardcover,374,Scholastic Press,09/14/08,6376780,"[District 12 Panem, Capitol Panem, Panem (Unit...",https://i.gr-assets.com/images/S/compressed.ph...,8.97
1,Harry Potter and the Order of the Phoenix,Harry Potter #5,J.K. Rowling,4.50,There is a door at the end of a silent corrido...,English,"[Fantasy, Young Adult, Fiction, Magic, Childre...","[Sirius Black, Draco Malfoy, Ron Weasley, Petu...",Paperback,870,Scholastic Inc.,09/28/04,2507623,[Hogwarts School of Witchcraft and Wizardry (U...,https://i.gr-assets.com/images/S/compressed.ph...,6.87
2,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,"[Young Adult, Fantasy, Romance, Vampires, Fict...","[Edward Cullen, Jacob Black, Laurent, Renee, B...",Paperback,501,"Little, Brown and Company",09/06/06,4964519,"[Forks Washington (United States), Phoenix Ari...",https://i.gr-assets.com/images/S/compressed.ph...,25.24
3,The Book Thief,,Markus Zusak,4.37,Librarian's note: An alternate cover edition c...,English,"[Historical Fiction, Fiction, Young Adult, His...","[Liesel Meminger, Hans Hubermann, Rudy Steiner...",Hardcover,552,Alfred A. Knopf,03/14/06,1834276,"[Molching (Germany), Germany]",https://i.gr-assets.com/images/S/compressed.ph...,12.87
4,Animal Farm,,George Orwell,3.95,Librarian's note: There is an Alternate Cover ...,English,"[Classics, Fiction, Dystopia, Fantasy, Literat...","[Snowball, Napoleon, Clover, Boxer, Old Major,...",Mass Market Paperback,141,Signet Classics,04/28/96,2740713,"[England, United Kingdom]",https://i.gr-assets.com/images/S/compressed.ph...,14.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34660,Attracted to Fire,,DiAnn Mills,4.14,Special Agent Meghan Connors' dream of one day...,English,"[Christian Fiction, Christian, Suspense, Roman...",[],Paperback,416,Tyndale House Publishers,October 1st 2011,2143,[West Texas (United States)],https://i.gr-assets.com/images/S/compressed.ph...,26.67
34661,Unbelievable,Port Fare #2,Sherry Gammon,4.16,Lilah Lopez Dreser's in town to take care of u...,English,"[Romance, Young Adult, Contemporary, Contempor...",[],Paperback,360,Wordpaintings Unlimited,April 11th 2013,1028,[Port Fare New York (United States)],https://i.gr-assets.com/images/S/compressed.ph...,16.59
34662,Marked,Soul Guardians #1,Kim Richardson,3.70,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,English,"[Fantasy, Young Adult, Paranormal, Angels, Rom...",[],Paperback,280,CreateSpace,March 18th 2011,6674,[],https://i.gr-assets.com/images/S/compressed.ph...,26.36
34663,Wayward Son,,Tom Pollack,3.85,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,English,"[Fiction, Mystery, Historical Fiction, Adventu...",[],Paperback,507,Cascada Productions,September 1st 2011,238,[],https://i.gr-assets.com/images/S/compressed.ph...,19.92


In [18]:
data = df.to_json(orient='index')
json_genres = json.dumps(genres)
json_setting = json.dumps(setting)
json_languages = json.dumps(languages)
json_authors = json.dumps(authors)

In [19]:
with open('Output/books.json', 'w') as file:
    file.write(data)

with open('Output/genres.json', 'w') as file:
    file.write(json_genres)

with open('Output/setting.json', 'w') as file:
    file.write(json_setting)

with open('Output/languages.json', 'w') as file:
    file.write(json_languages)

with open('Output/authors.json', 'w') as file:
    file.write(json_authors)

Tests

In [15]:
# for mset in df.setting:
#     for item in mset:
#         if re.search("\(Austria(.)*\)", item):
#             print(mset)
#             break

In [25]:
with open('Output/Test.json', 'w') as file:
    file.write(df.loc[0].to_json(orient='index'))